In [62]:
import pandas
import numpy as np
from scipy.stats import boxcox, boxcox_normmax
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV, train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Ridge, ElasticNet
from sklearn.svm import SVR
from sklearn.cluster import KMeans
pandas.set_option('display.max_columns', None)

data_train = pandas.read_csv('train.csv', index_col = 'Id')
data_test = pandas.read_csv('test.csv', index_col = 'Id')
data_train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [2]:
def print_nan(X):
    counts = X.count()
    length = X.shape[0]
    for i in counts[counts < length].index:
        print(i, ':', '%.3f' % (counts[i] / length))
    return
        
print('\nTrain')
print_nan(data_train)
print('\nTest')
print_nan(data_test)


Train
LotFrontage : 0.823
Alley : 0.062
MasVnrType : 0.995
MasVnrArea : 0.995
BsmtQual : 0.975
BsmtCond : 0.975
BsmtExposure : 0.974
BsmtFinType1 : 0.975
BsmtFinType2 : 0.974
Electrical : 0.999
FireplaceQu : 0.527
GarageType : 0.945
GarageYrBlt : 0.945
GarageFinish : 0.945
GarageQual : 0.945
GarageCond : 0.945
PoolQC : 0.005
Fence : 0.192
MiscFeature : 0.037

Test
MSZoning : 0.997
LotFrontage : 0.844
Alley : 0.073
Utilities : 0.999
Exterior1st : 0.999
Exterior2nd : 0.999
MasVnrType : 0.989
MasVnrArea : 0.990
BsmtQual : 0.970
BsmtCond : 0.969
BsmtExposure : 0.970
BsmtFinType1 : 0.971
BsmtFinSF1 : 0.999
BsmtFinType2 : 0.971
BsmtFinSF2 : 0.999
BsmtUnfSF : 0.999
TotalBsmtSF : 0.999
BsmtFullBath : 0.999
BsmtHalfBath : 0.999
KitchenQual : 0.999
Functional : 0.999
FireplaceQu : 0.500
GarageType : 0.948
GarageYrBlt : 0.947
GarageFinish : 0.947
GarageCars : 0.999
GarageArea : 0.999
GarageQual : 0.947
GarageCond : 0.947
PoolQC : 0.002
Fence : 0.199
MiscFeature : 0.035
SaleType : 0.999


In [161]:
data_train = pandas.read_csv('train.csv', index_col = 'Id')
data_test = pandas.read_csv('test.csv', index_col = 'Id')
data_train.drop(data_train[(data_train['GrLivArea']>4000) & (data_train['SalePrice']<300000)].index, inplace = True)
data_train.drop(data_train[(data_train['LotArea']>100000) | (data_train['SalePrice']>600000)].index, inplace = True)

def gen_features(X):
    X = X.copy()
    X['HouseAge'] = X['YrSold'].map(int) - X['YearBuilt']
    X['RemodAge'] = X['YrSold'].map(int) - X['YearRemodAdd']
    X['GarageAge'] = X['YrSold'].map(int) - X['GarageYrBlt']
    X['TotalBsmtSF'].fillna(value = 0, inplace = True)
    X['TotalSF'] = X['TotalBsmtSF'] + X['1stFlrSF'] + X['2ndFlrSF']
    # Linear dependence on new features
    X.drop(['YearBuilt'], axis = 1, inplace = True)
    X.drop(['YearRemodAdd'], axis = 1, inplace = True)
    X.drop(['GarageYrBlt'], axis = 1, inplace = True)
    # Linear dependence on BsmtFinSF1, BsmtFinSF2, BsmtFinUnfSF
    X.drop(['TotalBsmtSF'], axis = 1, inplace = True)
    # Approximately equal TotalBsmtSF
    X.drop(['1stFlrSF'], axis = 1, inplace = True)
    # Linear dependence on GrLivArea and 1stFlrSF
    X.drop(['2ndFlrSF'], axis = 1, inplace = True)
    low_variance_categories = [col for col in X_train.select_dtypes(exclude='number').columns if 1 - sum(X_train[col] == (X_train[col].mode()[0]))/len(X_train[col]) < 0.03]
    X.drop(low_variance_categories, axis = 1, inplace = True)
    return X
    
def fill_nan(X):
    X = X.copy()
    X['LotFrontage'] = X.groupby('Neighborhood')['LotFrontage'].apply(lambda x: x.fillna(x.median()))
    for column in ('GarageYrBlt', 'GarageArea', 'GarageCars', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'MasVnrArea'):
        X[column].fillna(value = 0, inplace = True)
    for column in ('MSZoning', 'Electrical', 'KitchenQual', 'Exterior1st', 'Exterior2nd', 'SaleType'):
        X[column].fillna(value = X[column].mode()[0], inplace = True)
    X['Functional'].fillna(value = 'Typ', inplace = True)
    X.fillna(value = 'NA', inplace = True)
    return X

def convert_features(X):
    X = X.copy()
    NumFeatures = ['MSSubClass', 'OverallQual', 'OverallCond', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'MoSold', 'YrSold']
    for column in NumFeatures:
        X[column] = X[column].map(str)
    return X

def clean_data(X):                                
    X = X.copy()
    X = fill_nan(X)
    X = convert_features(X)
    # X.drop('Utilities', axis = 1, inplace = True)
    return X

def normalize_data(X_train, X_test):
    X_train = X_train.copy()
    X_test = X_test.copy()
    numeric_features = X_train.dtypes[X_train.dtypes != 'object'].index
    shift = 1
    for column in numeric_features: 
        if ((column == 'GarageAge') or (column == 'HouseAge') or (column == 'RemodAge')):
            continue
        lam = boxcox_normmax(X_train[column] + shift)
        X_train[column] = X_train[column].map(lambda x: boxcox(x + shift, lam))
        X_test[column] = X_test[column].map(lambda x: boxcox(x + shift, lam))
    return X_train, X_test

data_train = clean_data(data_train)
data_test = clean_data(data_test)

X_train = data_train.drop(['SalePrice'], axis = 1)
y_train = data_train['SalePrice']
X_test = data_test
index = X_test.index

X_train_init = X_train.copy()
X_test_init = X_test.copy()
y_train_init = y_train.copy()
X_train, X_test = normalize_data(gen_features(X_train), gen_features(X_test))
y_train = np.log1p(y_train) 

def vectorize_data(X_train, X_test):
    X_train = X_train.copy()
    X_test = X_test.copy()    
    #nonnumeric_features = X_train.dtypes[X_train.dtypes == "object"].index
    A = X_train.copy()
    B = X_test.copy()
    for column in X_train.columns:
        le = LabelEncoder()
        ohe = OneHotEncoder(sparse = False)
        ohe.fit(le.fit_transform(pandas.Series(pandas.concat([X_train[column], X_test[column]]))).reshape(-1, 1))
        col_train = pandas.DataFrame(ohe.transform(le.transform(X_train[column]).reshape(-1, 1)))
        col_train.columns = [str(column) + str(int(i)) for i in range(col_train.shape[1])]
        col_test = pandas.DataFrame(ohe.transform(le.transform(X_test[column]).reshape(-1, 1)))
        col_test.columns = col_train.columns
        col_train.index = A.index
        col_test.index = B.index
        A = pandas.concat([A, col_train], axis = 1)
        B = pandas.concat([B, col_test], axis = 1)
    A.drop(X_train.columns, axis = 1, inplace = True)
    B.drop(X_train.columns, axis = 1, inplace = True)
    return A, B

def transform_data(X_train, X_test):
    train_index = X_train.index
    test_index = X_test.index
    scaler = StandardScaler()
    X_train = X_train.copy()
    X_test = X_test.copy()
    X_train_transformed = pandas.DataFrame(scaler.fit_transform(X_train.values), columns = X_train.columns, index = X_train.index)
    X_test_transformed = pandas.DataFrame(scaler.transform(X_test.values), columns = X_test.columns, index = X_test.index)
    return X_train_transformed, X_test_transformed

def vec_transform(X_train, X_test):
    X_train = X_train.copy()
    X_test = X_test.copy()
    X_train_num = X_train.select_dtypes(include = 'number')
    X_test_num = X_test[X_train_num.columns]
    X_train_obj =  X_train.select_dtypes(exclude = 'number')
    X_test_obj = X_test[X_train_obj.columns]
    X_train_num, X_test_num = transform_data(X_train_num, X_test_num)
    X_train_obj, X_test_obj = vectorize_data(X_train_obj, X_test_obj)
    return pandas.concat([X_train_num, X_train_obj], axis = 1), pandas.concat([X_test_num, X_test_obj], axis = 1)

X_train_transformed, X_test_transformed = vec_transform(X_train, X_test)

def clusterize(X_train, X_test):
    kmeans = KMeans(n_clusters = 30, init = 'k-means++', random_state = 42)
    ohe = OneHotEncoder(sparse = False)
    X_train = X_train.copy()
    X_test = X_test.copy()
    train = kmeans.fit_predict(X_train)
    test = kmeans.predict(X_test)
    train = pandas.DataFrame(ohe.fit_transform(train.reshape(-1, 1)), index = X_train.index)
    test = pandas.DataFrame(ohe.transform(test.reshape(-1, 1)), index = X_test.index)
    X_train = pandas.concat([X_train, train], axis = 1)
    X_test = pandas.concat([X_test, test], axis = 1)
    return X_train, X_test

X_train_clust, X_test_clust = clusterize(X_train_transformed, X_test_transformed)
print('train, transformed, clust', X_train.shape, X_train_transformed.shape, X_train_clust.shape)
print('test, transformed, clust', X_test.shape, X_test_transformed.shape, X_test_clust.shape)

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:3010: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:5256: RuntimeWarning: invalid value encountered in less
  x = np.where(x < 1.0, x, 1.0)  # if x > 1 then return 1.0


train, transformed, clust (1450, 71) (1450, 373) (1450, 403)
test, transformed, clust (1459, 71) (1459, 373) (1459, 403)


In [163]:
grid = {'n_estimators' : [2000, 3000], 
        'learning_rate' : [0.005, 0.01, 0.02], 
        'subsample' : [0.2, 0.3, 0.5], 
        'max_features' : [None, 'auto', 0.2, 0.3, 0.5]}
cv = KFold(n_splits = 5, shuffle = True, random_state = 241)
gbc = GradientBoostingRegressor(random_state = 241, loss = 'huber')
gs = GridSearchCV(gbc, grid, scoring = 'neg_mean_squared_error', cv = cv)
gs.fit(X = X_train_clust, y = y_train)
print(gs.best_params_)

{'learning_rate': 0.01, 'max_features': 0.3, 'n_estimators': 3000, 'subsample': 0.3}


In [156]:
def gbccalc_acc(X_train, y_train, scoring = 'r2'):
    cv = KFold(n_splits = 5, shuffle = True, random_state = 241)
    gbc = GradientBoostingRegressor(n_estimators = 3000, learning_rate = 0.01, subsample = 0.3, max_features = 0.3, random_state = 241)
    scores = np.mean(cross_val_score(estimator = gbc, X = X_train, y = y_train, cv = cv, scoring=scoring, n_jobs = -1))
    return scores

print("%.4f" % gbccalc_acc(X_train_transformed, y_train))
print("%.4f" % gbccalc_acc(X_train_clust, y_train))

0.9160
0.9151


In [127]:
grid = {'alpha' : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]}
cv = KFold(n_splits = 5, shuffle = True, random_state = 241)
ridge = Ridge()
gs = GridSearchCV(ridge, grid, scoring = 'neg_mean_squared_error', cv = cv, n_jobs = -1)
gs.fit(X = X_train_transformed, y = y_train)
print(gs.best_params_)

{'alpha': 100}


In [136]:
def ridgecalc_acc(X_train, y_train, scoring = 'r2'):
    cv = KFold(n_splits = 5, shuffle = True, random_state = 241)
    ridge = Ridge(alpha = 100)
    scores = np.mean(cross_val_score(estimator = ridge, X = X_train, y = y_train, cv = cv, scoring=scoring, n_jobs = -1))
    return scores

print("%.4f" % ridgecalc_acc(X_train_transformed, y_train))
print("%.4f" % ridgecalc_acc(X_train_clust, y_train_norm))


0.9071
0.9017
-519111154.9290
-0.0152


In [160]:
hp = GradientBoostingRegressor(n_estimators = 3000, learning_rate = 0.01, subsample = 0.3, max_features = 0.3, random_state = 241)
hp.fit(X_train_transformed, y_train)
y_test = pandas.DataFrame(data = {'SalePrice' : hp.predict(X_test_transformed)})
y_test = np.expm1(y_test)
y_test.index = index
y_test.to_csv('prices_predict_gb.csv')

In [158]:
hp = GradientBoostingRegressor(n_estimators = 3000, learning_rate = 0.01, subsample = 0.3, max_features = 0.3, random_state = 241)
hp.fit(X_train_clust, y_train)
y_test = pandas.DataFrame(data = {'SalePrice' : hp.predict(X_test_clust)})
y_test = np.expm1(y_test)
y_test.index = index
y_test.to_csv('prices_predict_gb_clust.csv')

In [101]:
hp = Ridge(alpha = 100)
hp.fit(X_train_transformed, y_train)
y_test = pandas.DataFrame(data = {'SalePrice' : hp.predict(X_test_transformed)})
y_test = np.expm1(y_test)
y_test.index = index
y_test.to_csv('prices_predict_ridge.csv')

In [ ]:
hp = Ridge(alpha = 100)
hp.fit(X_train_transformed, y_train)
y_test = pandas.DataFrame(data = {'SalePrice' : hp.predict(X_test_transformed)})
y_test = np.expm1(y_test)
y_test.index = index
y_test.to_csv('prices_predict_ridge_clust.csv')

In [153]:
from sklearn.linear_model import LinearRegression

def lrcalc_acc(X_train, y_train, scoring = 'r2'):
    cv = KFold(n_splits = 5, shuffle = True, random_state = 241)
    lr = LinearRegression()
    scores = np.mean(cross_val_score(estimator = lr, X = X_train, y = y_train, cv = cv, scoring=scoring))
    return scores

print("%.4f" % lrcalc_acc(X_train_transformed, y_train))
print("%.4f" % lrcalc_acc(X_train_clust, y_train))


0.8875
0.8955
0.8955
-42760624173457514094919680.0000
0.8816
0.8759
-26223669143755915274485760.0000


In [167]:
from sklearn.linear_model import HuberRegressor
grid = {'alpha' : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]}
cv = KFold(n_splits = 5, shuffle = True, random_state = 241)
hr = HuberRegressor()
gs = GridSearchCV(hr, grid, scoring = 'neg_mean_squared_error', cv = cv)
gs.fit(X = X_train_norm_transformed, y = y_train_norm)
print(gs.best_params_)

{'alpha': 1000}


In [40]:
def hrcalc_acc(X_train, y_train, scoring = 'r2'):
    cv = KFold(n_splits = 5, shuffle = True, random_state = 241)
    hr = HuberRegressor(alpha = 1000)#(n_estimators = 3000, learning_rate = 0.01, subsample = 0.3, max_features = 0.3, random_state = 241)
    scores = np.mean(cross_val_score(estimator = hr, X = X_train, y = y_train, cv = cv, scoring=scoring))
    return scores

print("%.4f" % hrcalc_acc(X_train_transformed, y_train))
print("%.4f" % hrcalc_acc(X_train_clust, y_train))

0.7234
0.7290
0.7290
-0.0306
-31.4269
-3.2639
0.9064


In [172]:
grid = {'alpha' : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000],
        'l1_ratio' : [0, 0.2, 0.4, 0.5, 0.6, 0.8, 1.0]}
cv = KFold(n_splits = 5, shuffle = True, random_state = 241)
enr = ElasticNet()
gs = GridSearchCV(enr, grid, scoring = 'neg_mean_squared_error', cv = cv)
gs.fit(X = X_train_norm_transformed, y = y_train_norm)
print(gs.best_params_)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increa

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increa

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


{'alpha': 0.01, 'l1_ratio': 0.4}


In [173]:
def enrcalc_acc(X_train, y_train, scoring = 'r2'):
    cv = KFold(n_splits = 5, shuffle = True, random_state = 241)
    enr = ElasticNet(alpha = 0.01, l1_ratio = 0.4)
    scores = np.mean(cross_val_score(estimator = enr, X = X_train, y = y_train, cv = cv, scoring=scoring))
    return scores

print("%.4f" % enrcalc_acc(X_train_transformed, y_train))
print("%.4f" % enrcalc_acc(X_train_clust, y_train))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increa

0.9165


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increa

0.9167


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increa

0.9167


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increa

0.8984


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increa

0.8723
0.8772
0.9185


In [175]:
from sklearn.linear_model import Lasso
grid = {'alpha' : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]}
cv = KFold(n_splits = 5, shuffle = True, random_state = 241)
lr = Lasso()
gs = GridSearchCV(lr, grid, scoring = 'neg_mean_squared_error', cv = cv)
gs.fit(X = X_train_norm_transformed, y = y_train_norm)
print(gs.best_params_)

{'alpha': 0.001}


In [177]:
def lrcalc_acc(X_train, y_train, scoring = 'r2'):
    cv = KFold(n_splits = 5, shuffle = True, random_state = 241)
    lr = Lasso(alpha = 0.001)
    scores = np.mean(cross_val_score(estimator = lr, X = X_train, y = y_train, cv = cv, scoring=scoring))
    return scores

print("%.4f" % lrcalc_acc(X_train_transformed, y_train))
print("%.4f" % lrcalc_acc(X_train_clust, y_train))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increa

0.8960


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increa

0.8918


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increa

0.8918


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increa

0.8917


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increa

0.9119
0.9154
0.9099


In [197]:
from sklearn.linear_model import SGDRegressor
grid = {'alpha' : [0.001, 0.01, 0.1, 1, 10, 100, 1000],
        'penalty' : [None, 'l2', 'l1', 'elasticnet'],
        'l1_ratio' : [0, 0.2, 0.4, 0.5, 0.6, 0.8, 1.0],
        'shuffle' : [True, False],
        'learning_rate' : ['constant', 'optimal', 'invscaling'],
        'max_iter' : [5, 1000]}
cv = KFold(n_splits = 5, shuffle = True, random_state = 241)
sgd = SGDRegressor(random_state = 42)
gs = GridSearchCV(sgd, grid, scoring = 'neg_mean_squared_error', cv = cv)
gs.fit(X = X_train_norm_transformed, y = y_train_norm)
print(gs.best_params_)

{'alpha': 0.1, 'l1_ratio': 0, 'learning_rate': 'invscaling', 'max_iter': 1000, 'penalty': 'l2', 'shuffle': False}


In [199]:
def sgdcalc_acc(X_train, y_train, scoring = 'r2'):
    cv = KFold(n_splits = 5, shuffle = True, random_state = 241)
    sgd = SGDRegressor(random_state = 42, alpha = 0.1, l1_ratio = 0, learning_rate = 'invscaling', max_iter = 1000, penalty = 'l2')
    scores = np.mean(cross_val_score(estimator = sgd, X = X_train, y = y_train, cv = cv, scoring=scoring))
    return scores

print("%.4f" % sgdcalc_acc(X_train_transformed, y_train))
print("%.4f" % sgdcalc_acc(X_train_clust, y_train))

-103500089314317371965440.0000
-90465917214951806599168.0000
-90465917214951806599168.0000
0.9079
-2326573173249807332376778215784448.0000
-211505577423876547393496285184.0000
0.9009


In [17]:
X_train_models, X_train_stack, y_train_models, y_train_stack = train_test_split(X_train_clust, y_train_norm, test_size = 0.3, random_state = 241, shuffle = True)

gbc = GradientBoostingRegressor(n_estimators = 3000, learning_rate = 0.01, subsample = 0.3, max_features = 0.3, random_state = 241)
ridge = Ridge(alpha = 100)
enr = ElasticNet(alpha = 0.01, l1_ratio = 0.4)
gbc.fit(X_train_models, y_train_models)
ridge.fit(X_train_models, y_train_models)
enr.fit(X_train_models, y_train_models)

y_stack = pandas.concat([pandas.Series(gbc.predict(X_train_stack)), pandas.Series(ridge.predict(X_train_stack)), pandas.Series(enr.predict(X_train_stack))], axis = 1)

In [49]:
grid = {'C' : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000], 
        'gamma' : ['auto', 0.01, 0.02, 0.03, 0.05, 0.1, 0.2]}
cv = KFold(n_splits = 5, shuffle = True, random_state = 241)
svr = SVR()
gs = GridSearchCV(svr, grid, scoring = 'neg_mean_squared_error', cv = cv)
gs.fit(X = y_stack, y = y_train_stack)
print(gs.best_params_)

{'C': 10000, 'gamma': 0.03}


In [50]:
def svrcalc_acc(X_train, y_train, scoring = 'r2'):
    cv = KFold(n_splits = 5, shuffle = True, random_state = 241)
    svr = SVR(C = 10000, gamma = 0.03)
    scores = np.mean(cross_val_score(estimator = svr, X = X_train, y = y_train, cv = cv, scoring=scoring))
    return scores

print("%.4f" % svrcalc_acc(y_stack, y_train_stack))

0.9149


In [20]:
gbc = GradientBoostingRegressor(n_estimators = 3000, learning_rate = 0.01, subsample = 0.3, max_features = 0.3, random_state = 241)
ridge = Ridge(alpha = 100)
enr = ElasticNet(alpha = 0.01, l1_ratio = 0.4)

#models = [gbc, ridge, enr]
#for model in models:
#    model.fit(X_train_models, y_train_models)
gbc.fit(X_train_models, y_train_models)
ridge.fit(X_train_models, y_train_models)
enr.fit(X_train_models, y_train_models)

svr = SVR(C = 10000, gamma = 0.03)
y_stack = pandas.concat([pandas.Series(gbc.predict(X_train_stack)), pandas.Series(ridge.predict(X_train_stack)), pandas.Series(enr.predict(X_train_stack))], axis = 1)
svr.fit(y_stack, y_train_stack)

y_stack_test = pandas.concat([pandas.Series(gbc.predict(X_test_clust)), pandas.Series(ridge.predict(X_test_clust)), pandas.Series(enr.predict(X_test_clust))], axis = 1)

y_test = pandas.DataFrame(data = {'SalePrice' : svr.predict(y_stack_test)})
y_test = np.expm1(y_test)
y_test.index = index
y_test.to_csv('prices_predict_stacking.csv')

In [27]:
coef = pandas.Series(ridge.coef_)
print(sum(coef != 0))
print(sum(coef == 0))

410
2


In [28]:
coef = pandas.Series(enr.coef_)
print(sum(coef != 0))
print(sum(coef == 0))

126
286
